In [5]:
#gpt attempt
import pandas as pd
import networkx as nx
from collections import defaultdict
import numpy as np

file_path = '/kaggle/input/train-txt/train.txt'
df = pd.read_csv(file_path, sep=r'\s+', names=['head','relation','tail'])

people = pd.Index(df['head']).union(df['tail'])
print("Train triples:", len(df), "| people:", len(people), "| relations:", df['relation'].nunique())

# --- parent->child backbone T ---
PARENT_REL = {'fatherOf','motherOf'}
CHILD_REL  = {'sonOf','daughterOf'}

T = nx.DiGraph()
T.add_nodes_from(people)

for h,r,t in df[['head','relation','tail']].itertuples(index=False):
    if r in PARENT_REL:
        T.add_edge(h,t)
    elif r in CHILD_REL:
        T.add_edge(t,h)

print("Backbone edges (parent->child):", T.number_of_edges())

# Generation estimate (component-wise)
gen = {}
for comp in nx.weakly_connected_components(T):
    sub = T.subgraph(comp)
    roots = [n for n in sub.nodes() if sub.in_degree(n)==0]
    for r in roots: gen[r]=0
    for node in nx.topological_sort(sub):
        if node not in gen:
            parents = list(sub.predecessors(node))
            gen[node] = (max(gen[p] for p in parents) + 1) if parents else 0

# Family id = backbone weak component id
comps = list(nx.weakly_connected_components(T))
family_id = {}
for i, comp in enumerate(comps):
    for n in comp:
        family_id[n] = i

print("Families (components):", len(comps),
      "| size range:", (min(len(c) for c in comps), max(len(c) for c in comps)))

# --- G_all: all relations as undirected weighted edges ---
G_all = nx.Graph()
G_all.add_nodes_from(people)

for h,r,t in df[['head','relation','tail']].itertuples(index=False):
    a,b = (h,t) if h < t else (t,h)
    if G_all.has_edge(a,b):
        G_all[a][b]['weight'] += 1
        G_all[a][b]['relations'].add(r)
    else:
        G_all.add_edge(a,b, weight=1, relations={r})

print("G_all edges:", G_all.number_of_edges(),
      "| mean weight:", np.mean([d['weight'] for _,_,d in G_all.edges(data=True)]))

# --- Build siblings and couples from T (needs 2-parent children) ---
parents2 = {c: tuple(sorted(T.predecessors(c))) for c in T.nodes()}
child2 = {c:p for c,p in parents2.items() if len(p)==2}

couple_children = defaultdict(set)
for c,(p1,p2) in child2.items():
    couple_children[(p1,p2)].add(c)

# full-sibs: share both parents
sib_pairs = set()
for (p1,p2), kids in couple_children.items():
    kids = sorted(kids)
    for i in range(len(kids)):
        for j in range(i+1,len(kids)):
            sib_pairs.add((kids[i], kids[j]))

print("Couples w/children:", len(couple_children),
      "| children w/2 parents:", len(child2),
      "| sibling pairs (derived):", len(sib_pairs))

# --- G_close: close-kin graph (parent-child + siblings + spouse) ---
G_close = nx.Graph()
G_close.add_nodes_from(people)

def add_w(u,v,w,etype):
    a,b = (u,v) if u < v else (v,u)
    if G_close.has_edge(a,b):
        G_close[a][b]['weight'] += w
        G_close[a][b]['types'].add(etype)
    else:
        G_close.add_edge(a,b, weight=w, types={etype})

# parent-child edges (strong)
for p,c in T.edges():
    add_w(p,c, w=3, etype='parent')

# sibling edges (strong)
for a,b in sib_pairs:
    add_w(a,b, w=2, etype='sibling')

# spouse edges inferred by co-parenting (moderate)
for (p1,p2) in couple_children.keys():
    add_w(p1,p2, w=2, etype='spouse')

print("G_close edges:", G_close.number_of_edges())

Train triples: 13821 | people: 14634 | relations: 28
Backbone edges (parent->child): 1642
Families (components): 50 | size range: (26, 27)
G_all edges: 7480 | mean weight: 1.8477272727272727
Couples w/children: 445 | children w/2 parents: 821 | sibling pairs (derived): 603
G_close edges: 2690


In [6]:
from networkx.algorithms.community import modularity

def run_louvain(G):
    try:
        return nx.community.louvain_communities(G, weight='weight', seed=42)
    except Exception as e:
        print("Louvain not available in this NetworkX:", e)
        raise

def run_lpa(G):
    # weighted async label propagation if available
    try:
        comms = list(nx.community.asyn_lpa_communities(G, weight='weight', seed=42))
        return comms
    except TypeError:
        comms = list(nx.community.asyn_lpa_communities(G, seed=42))
        return comms

def summarize_partition(G, comms, name):
    sizes = sorted([len(c) for c in comms], reverse=True)
    Q = modularity(G, comms, weight='weight')
    intra = 0
    for u,v,d in G.edges(data=True):
        w = d.get('weight',1)
        # membership test via map
    print(f"\n== {name} ==")
    print("communities:", len(comms))
    print("size stats (top5):", sizes[:5], "| min/median/max:", (min(sizes), int(np.median(sizes)), max(sizes)))
    print("modularity Q:", round(Q,4))
    return Q, sizes

def comm_map(comms):
    m = {}
    for i,c in enumerate(comms):
        for n in c:
            m[n]=i
    return m

# Run
parts = {}
for Gname, G in [('G_all', G_all), ('G_close', G_close)]:
    L = run_louvain(G); parts[(Gname,'louvain')] = L
    P = run_lpa(G);     parts[(Gname,'lpa')]     = P

    summarize_partition(G, L, f"{Gname} / Louvain")
    summarize_partition(G, P, f"{Gname} / LabelProp")


== G_all / Louvain ==
communities: 50
size stats (top5): [27, 27, 27, 27, 27] | min/median/max: (26, 26, 27)
modularity Q: 0.9793

== G_all / LabelProp ==
communities: 67
size stats (top5): [27, 27, 27, 27, 27] | min/median/max: (2, 26, 27)
modularity Q: 0.9567

== G_close / Louvain ==
communities: 50
size stats (top5): [27, 27, 27, 27, 27] | min/median/max: (26, 26, 27)
modularity Q: 0.9799

== G_close / LabelProp ==
communities: 296
size stats (top5): [17, 17, 14, 14, 12] | min/median/max: (2, 4, 17)
modularity Q: 0.7696


In [7]:
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

true = [family_id[n] for n in people]

for (Gname, alg), comms in parts.items():
    pred_map = comm_map(comms)
    pred = [pred_map[n] for n in people]
    nmi = normalized_mutual_info_score(true, pred)
    ari = adjusted_rand_score(true, pred)
    print(f"{Gname:7s} {alg:7s} | NMI={nmi:.4f} | ARI={ari:.4f} | #comms={len(comms)}")

G_all   louvain | NMI=1.0000 | ARI=1.0000 | #comms=50
G_all   lpa     | NMI=0.9868 | ARI=0.9713 | #comms=67
G_close louvain | NMI=1.0000 | ARI=1.0000 | #comms=50
G_close lpa     | NMI=0.8380 | ARI=0.4256 | #comms=296


In [8]:
def nuclear_coherence(comms):
    cm = comm_map(comms)
    ok = 0
    total = 0
    for (p1,p2), kids in couple_children.items():
        members = set(kids) | {p1,p2}
        total += 1
        labs = {cm[m] for m in members if m in cm}
        if len(labs) == 1:
            ok += 1
    return ok, total, ok/total if total else 0

for (Gname, alg), comms in parts.items():
    ok,total,rate = nuclear_coherence(comms)
    print(f"{Gname:7s} {alg:7s} | nuclear families kept intact: {ok}/{total} ({rate:.3f})")

G_all   louvain | nuclear families kept intact: 445/445 (1.000)
G_all   lpa     | nuclear families kept intact: 414/445 (0.930)
G_close louvain | nuclear families kept intact: 445/445 (1.000)
G_close lpa     | nuclear families kept intact: 247/445 (0.555)


In [9]:
import pandas as pd

def gen_span_stats(comms):
    spans = []
    for c in comms:
        g = [gen[n] for n in c]
        spans.append(max(g) - min(g) + 1)
    return pd.Series(spans)

for (Gname, alg), comms in parts.items():
    s = gen_span_stats(comms)
    print(f"\n{Gname} {alg} generation span:")
    print(s.describe())
    print("span value counts (top):")
    print(s.value_counts().sort_index().head(10))


G_all louvain generation span:
count    50.000000
mean      5.580000
std       0.784805
min       4.000000
25%       5.000000
50%       6.000000
75%       6.000000
max       7.000000
dtype: float64
span value counts (top):
4     4
5    18
6    23
7     5
Name: count, dtype: int64

G_all lpa generation span:
count    67.000000
mean      4.970149
std       1.413894
min       2.000000
25%       4.000000
50%       5.000000
75%       6.000000
max       7.000000
dtype: float64
span value counts (top):
2     8
3     2
4     8
5    20
6    24
7     5
Name: count, dtype: int64

G_close louvain generation span:
count    50.000000
mean      5.580000
std       0.784805
min       4.000000
25%       5.000000
50%       6.000000
75%       6.000000
max       7.000000
dtype: float64
span value counts (top):
4     4
5    18
6    23
7     5
Name: count, dtype: int64

G_close lpa generation span:
count    296.000000
mean       3.875000
std        1.414663
min        1.000000
25%        3.000000
50%       

In [10]:
def participation_coeff(G, comms):
    cm = comm_map(comms)
    pc = {}
    for n in G.nodes():
        if G.degree(n, weight='weight') == 0:
            pc[n] = 0.0
            continue
        k = defaultdict(float)
        for nbr, attr in G[n].items():
            w = attr.get('weight', 1.0)
            k[cm[nbr]] += w
        tot = sum(k.values())
        pc[n] = 1.0 - sum((x/tot)**2 for x in k.values())
    return pc

for (Gname, alg), comms in parts.items():
    G = G_all if Gname=='G_all' else G_close
    pc = participation_coeff(G, comms)
    top = sorted(pc.items(), key=lambda x: x[1], reverse=True)[:10]
    print(f"\nTop bridges by participation: {Gname} {alg}")
    print(top)


Top bridges by participation: G_all louvain
[('adam1073', 0.0), ('adam125', 0.0), ('adam1281', 0.0), ('adam198', 0.0), ('adam306', 0.0), ('adam359', 0.0), ('adam426', 0.0), ('adam474', 0.0), ('adam627', 0.0), ('adam719', 0.0)]

Top bridges by participation: G_all lpa
[('adam892', 0.5), ('elias865', 0.5), ('emma951', 0.5), ('johanna688', 0.5), ('karin651', 0.5), ('katharina861', 0.5), ('larissa1020', 0.5), ('laura851', 0.5), ('nico888', 0.5), ('noah1194', 0.5)]

Top bridges by participation: G_close louvain
[('adam1073', 0.0), ('adam125', 0.0), ('adam1281', 0.0), ('adam198', 0.0), ('adam306', 0.0), ('adam359', 0.0), ('adam426', 0.0), ('adam474', 0.0), ('adam627', 0.0), ('adam719', 0.0)]

Top bridges by participation: G_close lpa
[('angelina978', 0.78125), ('marcel214', 0.7733333333333333), ('marie215', 0.7733333333333333), ('adrian213', 0.7681660899653979), ('lena1265', 0.7681660899653979), ('marcel27', 0.743801652892562), ('oliver165', 0.743801652892562), ('patrick1290', 0.74380165289

In [11]:
from collections import deque

# Precompute ancestor depths per node within each family (fast enough here)
subT = {i: T.subgraph(comp).copy() for i, comp in enumerate(comps)}

def anc_depth_map(sub, node):
    d = {node: 0}
    q = deque([node])
    while q:
        x = q.popleft()
        for p in sub.predecessors(x):
            if p not in d:
                d[p] = d[x] + 1
                q.append(p)
    return d

anc = {}
for i in range(len(comps)):
    anc[i] = {n: anc_depth_map(subT[i], n) for n in subT[i].nodes()}

def relatedness(a, b):
    if family_id[a] != family_id[b]:
        return 0.0
    i = family_id[a]
    da = anc[i][a]; db = anc[i][b]
    common = set(da) & set(db)
    if not common:
        return 0.0
    smin = min(da[x] + db[x] for x in common)
    cmin = [x for x in common if da[x] + db[x] == smin]
    return len(cmin) * (0.5 ** smin)

# Demo: sample 10 random pairs inside same family and print score + hop distance on backbone
import random
random.seed(42)

pairs = []
fam0 = list(comps[0])
for _ in range(10):
    a,b = random.sample(fam0,2)
    pairs.append((a,b, relatedness(a,b)))

pairs_sorted = sorted(pairs, key=lambda x: x[2], reverse=True)
print("Sample relatedness scores (same family):")
for a,b,s in pairs_sorted:
    print(a,b,"score=",s)

Sample relatedness scores (same family):
marko1057 hannah1068 score= 0.25
laura1075 vanessa1060 score= 0.25
hannah1068 angelina1059 score= 0.125
tobias1076 jan1062 score= 0.125
maria1077 nora1056 score= 0.0
moritz1061 isabella1065 score= 0.0
isabella1065 nora1056 score= 0.0
olivia1054 isabella1065 score= 0.0
helena1070 tobias1076 score= 0.0
nina1063 moritz1061 score= 0.0
